# Image Processing and Analysis

- Optical Mark Recognition
- Face Emotion Detection
- Color Detection
- Background Removal
- Fruit Classifier
- Face Mask Detection
- Cookie Cutter Squid Game
- Green Light Red Light Squid Game
- Hand Distance Measurement
- Real-time Digit Classification

# Face Emotion Detection

## Modul yang diperlukan

In [3]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

## Persiapan Data

In [9]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [10]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [11]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed
                                image    label
0         images/train/happy/3578.jpg    happy
1        images/train/happy/16988.jpg    happy
2         images/train/happy/2666.jpg    happy
3         images/train/happy/5109.jpg    happy
4        images/train/happy/11981.jpg    happy
...                               ...      ...
28816  images/train/disgust/10112.jpg  disgust
28817  images/train/disgust/21668.jpg  disgust
28818   images/train/disgust/7049.jpg  disgust
28819   images/train/disgust/9716.jpg  disgust
28820   images/train/disgust/3561.jpg  disgust

[28821 rows x 2 columns]


In [12]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed
                                    image    label
0       images/validation/happy/23933.jpg    happy
1       images/validation/happy/24906.jpg    happy
2       images/validation/happy/18033.jpg    happy
3       images/validation/happy/15271.jpg    happy
4       images/validation/happy/26888.jpg    happy
...                                   ...      ...
7061  images/validation/disgust/20761.jpg  disgust
7062  images/validation/disgust/28710.jpg  disgust
7063  images/validation/disgust/23876.jpg  disgust
7064   images/validation/disgust/9460.jpg  disgust
7065  images/validation/disgust/35580.jpg  disgust

[7066 rows x 2 columns]


 ## Pemrosesan Awal Data

In [13]:
from tqdm.notebook import tqdm

In [14]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [15]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7066 [00:00<?, ?it/s]

## Normalisasi:


In [16]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

## Membangun Model Jaringan Neural

In [21]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'] )

##  Melatih Model

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.2252 - loss: 1.8466 - val_accuracy: 0.2583 - val_loss: 1.8125
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 283s 1s/step - accuracy: 0.2493 - loss: 1.8138 - val_accuracy: 0.2634 - val_loss: 1.7920
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 281s 1s/step - accuracy: 0.2640 - loss: 1.7685 - val_accuracy: 0.3262 - val_loss: 1.6595
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 280s 1s/step - accuracy: 0.3356 - loss: 1.6578 - val_accuracy: 0.4329 - val_loss: 1.4725
Epoch 5/100
 91/226 ━━━━━━━━━━━━━━━━━━━━ 2:43 1s/step - accuracy: 0.3923 - loss: 1.5393

## Model Menyimpan Model

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")